## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [2]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [5]:
# 讀取糖尿病資料集
diabetes = datasets.load_diabetes()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=4)

# 建立模型
gbr = GradientBoostingRegressor(random_state=7) #learning_rate=0.1, n_estimators=100, max_depth=3

In [7]:
# 先看看使用預設參數得到的結果，約為 的 MSE
gbr.fit(x_train, y_train)
y_pred = gbr.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

3669.1318234332707


In [10]:
# 設定要訓練的超參數組合
n_estimators = [100, 500, 1000]
max_depth = [1, 3, 5]
learning_rate = [0.01, 0.05, 0.1]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(gbr, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型


Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/ben/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    7.8s finished
/Users/ben/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [11]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -3149.767828 using {'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 500}


In [14]:
# 使用最佳參數重新建立模型
gbr_bestparam = GradientBoostingRegressor(max_depth=1,n_estimators=500, learning_rate=0.05, verbose=1)

# 訓練模型
gbr_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = gbr_bestparam.predict(x_test)

      Iter       Train Loss   Remaining Time 
         1        5862.7583            0.68s
         2        5695.1018            0.62s
         3        5542.5958            0.58s
         4        5396.5491            0.54s
         5        5263.8237            0.51s
         6        5136.5992            0.48s
         7        5021.0908            0.48s
         8        4909.7894            0.70s
         9        4809.1531            0.68s
        10        4712.0741            0.65s
        20        4016.0095            0.44s
        30        3594.6509            0.37s
        40        3317.7762            0.32s
        50        3129.2453            0.28s
        60        2997.6657            0.27s
        70        2902.3480            0.24s
        80        2832.7870            0.23s
        90        2780.6083            0.21s
       100        2736.9053            0.20s
       200        2478.5835            0.12s
       300        2339.3869            0.06s
       40

In [15]:
#從3669 -> 3183
print(metrics.mean_squared_error(y_test, y_pred))

3183.0836957275287
